In [4]:
def aa():
    """asd
    asdasd
    asdasdas
    asdasd
    """
    print("=================")



In [2]:
import autogen

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        # "model": ["gpt4", "gpt-4-32k", "gpt-4-32k-0314", "gpt-4-32k-v0314"],
        "model": ["gpt-35-turbo-16k-dep-001"]
    },
)

llm_config={
    "request_timeout": 600,
    "seed": 42,
    "config_list": config_list,
    "temperature": 0,
}

/home/awesomeyuer/miniconda3/envs/autogen-py311-env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# create an AssistantAgent instance named "assistant"
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config=llm_config,
)
# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    # human_input_mode="TERMINATE",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=5,
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    llm_config=llm_config,
    code_execution_config={"work_dir": "00_awesome_poc"},
    system_message="""Reply TERMINATE if the task has been solved at full satisfaction.
Otherwise, reply CONTINUE, or the reason why the task is not solved yet."""
)

In [12]:
import pandas as pd

csvFilePath = "SamplesData/Orders.csv"

df = pd.read_csv(csvFilePath,sep=',')
df

# df.to_json(orient="records", force_ascii=False)

,OrderId,OrderNo,CustomerNo,Product,Price
0,4f2afbc8507e4effb17e81dae7a4fe19,1,customer1,烤白薯,0.5
1,1aaa41cd9ddf46fa93aeea0478992746,2,customer2,酱香拿铁,3.1
2,2187eb0eedcf437bbc4cb310f51e9ed9,3,customer3,蒜香卡布奇诺,10.1
3,05e94bc32e9d446980b50d65d85e32a3,4,customer4,烤串,0.1
4,6d70527d4a1144f889a259b778ea9699,5,customer5,蒜香卡布奇诺,10.1
...,...,...,...,...,...
234,1c030c255027470d84f22260c84ad030,235,customer235,烤白薯,0.5
235,8971cc850dbf49cd87297bf3e91abc26,236,customer236,肉夹馍,1.1
236,64384e8fa49f4bb4b841ee6214430198,237,customer237,烤白薯,0.5
237,60ad593a3a2d4693ac4e65eb24c5e63b,238,customer238,驴肉火烧,7.3


# `AutoGen` 可能能部分解决 `数据不出境` 问题

## 营销活动策略-落地
1. 读取多个数据表，获得用户、销售、活动洞察
1. 总结洞察，形成新活动策略
1. 拆解策略（活动名，用户群，活动商品，活动折扣等）
1. 执行策略（生成短信/邮件，生成规则，~~发送~~）

## 运营活动落地：
1. 每日按照用户输入活动主题，选择用户群
1. 选择商品，匹配不同用户群
1. 为不同用户群-商品生成活动文案
1. 发送短信

## 我们在研究Autogen的时候，需要想想：
- 哪个Agent/角色负责哪一块？
- 有几个user_proxy，分别负责什么？


In [8]:
url = "http://192.168.0.105:5000/SamplesData/Orders.csv"

user_proxy.initiate_chat(
    assistant,
    # message="""Show me the YTD gain of 10 largest technology companies as of today.""",
    # message="""展示今天截至目前为止，10家最大的科技公司的年初至今收益。""",

    # succeed
    # message=f"""
    # 请分析订单数据文件: "{url}"(没有"quantity"字段) , 然后展示销量最多的商品即: "Product"
    # """,

    # succeed
    #  message=f"""
    # 请分析订单数据文件: "{url}", 获取最受欢迎的商品
    # """,

    # succeed
    #  message=f"""
    # 请分析订单数据文件: "{url}", 获取最不受欢迎的商品
    # """,

    # message=f"""
    # 请分析订单数据文件: "{url}"(没有"quantity"字段) , 哪个商品该促销?
    # """,

    # failed
    # message=f"""
    # 请分析订单数据文件: "{url}"(没有"quantity"字段) , 哪个商品(Product)该停产?
    # """,

    # succeed 订单最少的
    # message=f"""
    # 请分析订单数据文件: "{url}"(没有"quantity"字段) , 哪个商品(Product)该打折促销?
    # """,

    # message=f"""
    # 请分析订单数据文件: "{url}"(没有"quantity"字段) , 哪个昂贵的商品(Product)该打折促销?
    # """,

    message=f"""
    请分析订单数据文件: "{url}"(没有"quantity"字段) , 哪个昂贵的商品(Product)该打折促销? 
    """,
)

user_proxy (to assistant):


    请分析订单数据文件: "http://192.168.0.105:5000/SamplesData/Orders.csv"(没有"quantity"字段) , 哪个昂贵的商品(Product)该打折促销? 
    

--------------------------------------------------------------------------------
assistant (to user_proxy):

首先，我们需要下载并读取订单数据文件以获取所需信息。然后，我们可以计算每个商品的平均价格，并找到价格最高的商品。最后，我们可以检查该商品是否适合打折促销。

以下是解决这个问题的步骤和代码：

1. 下载并读取订单数据文件：
我们可以使用Python的`requests`库来下载文件，并使用`pandas`库来读取CSV文件。

```python
# filename: download_and_read_orders.py
import requests
import pandas as pd

# 下载订单数据文件
url = "http://192.168.0.105:5000/SamplesData/Orders.csv"
response = requests.get(url)
file_path = "Orders.csv"

# 将文件保存到本地
with open(file_path, "wb") as file:
    file.write(response.content)

# 读取CSV文件
df = pd.read_csv(file_path)
```

2. 计算每个商品的平均价格：
我们可以使用`groupby`函数按商品分组，并计算每个商品的平均价格。

```python
# filename: calculate_average_price.py

# 按商品分组并计算平均价格
average_price = df.groupby("Product")["Price"].mean()
```

3. 找到价格最高的商品：
我们可以使用`idxmax`函数找到平均价格最高的商品。

```python
# filename: find